# NonlinearModel
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
const source_name = "NonlinearModel";

In [ ]:
username = ""
task = ""

In [ ]:
using LightGBM
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("../../TrainingAlphas/Ensemble/ItemMetadata.ipynb");

In [ ]:
function compute_alpha(source)
    params = read_params(source)
    base_features =
        reduce(hcat, read_recommendee_alpha(x, "all").rating for x in params["alphas"])
    metadata_features = get_item_metadata_features(
        convert.(Int32, fill(1, num_items())),
        convert.(Int32, collect(1:num_items())),
        get_recommendee_split("implicit"),
    )
    X = hcat(base_features, metadata_features)

    preds = convert.(Float32, vec(predict(params["model"], X)))
    write_recommendee_alpha(preds, source)
end;

In [ ]:
function compute_explicit_alpha()
    source = "$task/Explicit"
    alphas = read_params(source)["alphas"]
    preds = zeros(Float32, num_items())
    for alpha in alphas
        preds += read_recommendee_alpha(alpha, "all").rating
    end
    write_recommendee_alpha(preds, source)
end;

In [ ]:
compute_alpha("$task/NonlinearExplicit")

In [ ]:
compute_alpha("$task/NonlinearImplicit")

In [ ]:
compute_explicit_alpha()